In [ ]:
from air2stream.native import a2s

In [ ]:
a2s

In [ ]:
cc = a2s.commondata

# Main input
cc.name = "Switzerland"        # name of the river/basin/region
cc.air_station = "SIO"              # name/ID of the air temperature station
cc.water_station = 2011            # name/ID of the water temperature station
cc.series = "c"                # type of series: c=continuous series, m=mean year
cc.time_res = "1d"                # time resolution: 1d=daily, nw=n weeks (n=1,2,...), 1m=monthly
cc.version = 8               # version: 3,4,5,7,8 parameters
cc.Tice_cover = 0                # Threshold temperature for ice formation
cc.fun_obj = "RMS"                # objective function: RMS (Root Mean Square Error), NSE (Nash-Sutcliffe Efficiency Index), KGE (Kling-Gupta Efficiency Index)
cc.mod_num = "CRN"              # mod_num : numerical method used to solve the model equation EUL (Euler Explicit), RK2 (Runge-Kutta 2), RK4 (Runge-Kutta 4), CRN (Crank Nicolson). CRN is the suggested choice.
cc.runmode = "PSO"                # optimization algorithm: PSO (Particle Swarm Optimization) or RANSAM (Random Sampling)
cc.prc = 0.60            # minimum percentage of data: 0...1. E.g., when using 1m time resolution, the monthly average is set to NaN when available data during one month are less than this percentage (60% in this case)    
cc.n_run = 500                # n_run: number of iterations
cc.mineff_index = -999            # minimum efficiency index (i.e., RMS, NSE, KGE). All model realization with efficiency index greater than this threshold are saved in file 0_...


In [ ]:
cc.station = cc.air_station + b'_' + cc.water_station

In [ ]:
cc.station

In [ ]:
cc.n_particles = 500
cc.c1 = 2
cc.c2 = 2
cc.wmax = 0.9
cc.wmin = 0.4


In [ ]:
import numpy as np

In [ ]:
a2s.init_param_arrays()

In [ ]:
cc.parmin = np.array([-5, -5, -5, -1, 0, 0, 0, -1], dtype=float)
cc.parmax = np.array([15, 1.5, 5, 1, 20, 10, 1, 5], dtype=float)

In [ ]:
import pandas as pd

In [ ]:
input_table = None
with open('../Switzerland/SIO_2011_cc.txt') as f:
    input_table = pd.read_table(f, header=None)

In [ ]:
input_table

In [ ]:
first_year = input_table[0][4]

In [ ]:
first_year_data = input_table.loc[input_table[0] < (first_year+1)].copy()

In [ ]:
first_year_data[0][:]=first_year-1

In [ ]:
first_year_data

In [ ]:
ext_input_table = pd.concat([first_year_data, input_table])

In [ ]:
n = len(ext_input_table)

In [ ]:
a2s.init_input_arrays(n)

In [ ]:
cc.date[:,0] = ext_input_table[0]

In [ ]:
cc.date[:,1] = ext_input_table[1]

In [ ]:
cc.date[:,2] = ext_input_table[2]

In [ ]:
cc.date

In [ ]:
cc.tair = ext_input_table[3]

In [ ]:
cc.twat_obs = ext_input_table[4]

In [ ]:
cc.q = ext_input_table[5]

In [ ]:
cc.q

In [ ]:
q = input_table[5].copy()

In [ ]:
q_positive = (q[q >= 0])
q_media = q_positive.mean() # code says medha (median?) but calcualted mean.

In [ ]:
cc.qmedia = q_media
cc.n_q = len(q_positive)

In [ ]:
cc.time_res

In [ ]:
# WARNING: this is supposed to be called ONCE only...
a2s.aggregation()

In [ ]:
a2s.statis()

In [ ]:
cc.std_obs

In [ ]:
cc.mean_obs

In [ ]:
cc.qmedia

In [ ]:
cc.par_best

In [ ]:
cc.finalfit

In [ ]:
a2s.pso_array_init()

In [ ]:
cc.par_best

In [ ]:
cc.finalfit

In [ ]:
cc.mineff_index

In [ ]:
cc.twat_obs

In [ ]:
cc.twat_mod

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(cc.twat_obs, cc.twat_mod, 'r.')

In [ ]:

import numpy as np, pandas as pd, xarray as xr
from typing import Sequence

def plot_two_arrays(
    a: np.ndarray,
    b: np.ndarray,
    names: Sequence[str] = None,
    xlab="time",
    ylab=None,
    title=None,
    **kwargs
):
    _, ax = plt.subplots(figsize=(16, 9))
    if names is None:
        names = ["First series", "Second series"]
    ax.plot(np.arange(len(a)), a, linewidth=2, label=names[0])
    ax.plot(np.arange(len(a)), b, linewidth=2, label=names[1])
    ax.legend()
    if xlab is not None:
        ax.set_xlabel(xlab)
    if ylab is not None:
        ax.set_ylabel(ylab)
    if title is not None:
        ax.set_title(title)
    plt.show()

In [ ]:
obs = cc.twat_obs
mod = cc.twat_mod


In [ ]:
plot_two_arrays(obs[:], cc.twat_mod)